In [1]:
import torch
from atom3d.datasets import LMDBDataset
from atom3d.splits.splits import split_randomly
import sys
sys.path.append('../')
import gvp
import gvp.atom3d
import torch_geometric

sys.path.insert(0, '/data/psample/repos/genie')
from genie.utils import  visualize

def compute_frenet_frames(x, mask, eps=1e-10):
    # x: [b, n_res, 3]

    t = x[:, 1:] - x[:, :-1]
    t_norm = torch.sqrt(eps + torch.sum(t ** 2, dim=-1))
    t = t / t_norm.unsqueeze(-1)

    b = torch.cross(t[:, :-1], t[:, 1:])
    b_norm = torch.sqrt(eps + torch.sum(b ** 2, dim=-1))
    b = b / b_norm.unsqueeze(-1)

    n = torch.cross(b, t[:, 1:])

    tbn = torch.stack([t[:, 1:], b, n], dim=-1)

    # TODO: recheck correctness of this implementation
    rots = []
    for i in range(mask.shape[0]):
        rots_ = torch.eye(3).unsqueeze(0).repeat(mask.shape[1], 1, 1)
        length = torch.sum(mask[i]).int()
        rots_[1:length-1] = tbn[i, :length-2]
        rots_[0] = rots_[1]
        rots_[length-1] = rots_[length-2]
        rots.append(rots_)
    rots = torch.stack(rots, dim=0).to(x.device)

    return rots

In [3]:
device = 'cuda'
data_path = '../data/paul_PPbind/test/'
# data_path = '../data/paul_DB5_lmdb_random_chain_is_always_2/'
dataset = LMDBDataset(data_path, transform=gvp.atom3d.PPBindingTransform(plm=0, device=device))
trainset, valset, testset = split_randomly(dataset)

dl = torch_geometric.loader.DataLoader(dataset=dataset,
                                            num_workers=0,
                                            batch_size=8,
                                            shuffle=False)

2023-02-25 22:29:23,190 INFO 2912: Splitting dataset with 30472 entries.
2023-02-25 22:29:23,192 INFO 2912: Size of the training set: 24378
2023-02-25 22:29:23,192 INFO 2912: Size of the validation set: 3047
2023-02-25 22:29:23,192 INFO 2912: Size of the test set: 3047


In [4]:
x = trainset[0][0].x
x = x.unsqueeze(0)
mask = torch.ones(x.shape[1]).unsqueeze(0)
frames = compute_frenet_frames(x, mask)
frames = frames.reshape(frames.shape[1], -1)

(x.shape, mask.shape)

(torch.Size([1, 234, 3]), torch.Size([1, 234]))

In [31]:

frames.shape

torch.Size([234, 9])

RuntimeError: shape '[234, 6]' is invalid for input of size 2106

In [22]:
visualize.plot_protein_coords(frames[:, :, 2])

In [2]:
device = 'cuda'
data_path = '../data/paul_PPbind/test/'
# data_path = '../data/paul_DB5_lmdb_random_chain_is_always_2/'
dataset = LMDBDataset(data_path, transform=gvp.atom3d.PPBindingTransform(plm=0, use_frenet_serret_frames=True, device=device))
trainset, valset, testset = split_randomly(dataset)

dl = torch_geometric.loader.DataLoader(dataset=dataset,
                                            num_workers=0,
                                            batch_size=8,
                                            shuffle=False)

2023-02-25 23:09:51,464 INFO 3747: Splitting dataset with 30472 entries.
2023-02-25 23:09:51,466 INFO 3747: Size of the training set: 24378
2023-02-25 23:09:51,466 INFO 3747: Size of the validation set: 3047
2023-02-25 23:09:51,467 INFO 3747: Size of the test set: 3047


In [3]:
x = trainset[0][0].x
x.shape

torch.Size([1, 234, 3]) torch.Size([1, 234])
torch.Size([1, 234, 3, 3])


torch.Size([234, 9])

In [4]:
x

tensor([[-0.1722, -0.6434, -0.7459,  ..., -0.9633,  0.2684, -0.0091],
        [-0.1722, -0.6434, -0.7459,  ..., -0.9633,  0.2684, -0.0091],
        [ 0.9139, -0.2524,  0.3180,  ..., -0.3633, -0.1590,  0.9180],
        ...,
        [ 0.4329, -0.8974, -0.0860,  ...,  0.5748,  0.3482, -0.7405],
        [-0.7731, -0.3627, -0.5203,  ...,  0.0058,  0.8163, -0.5776],
        [-0.7731, -0.3627, -0.5203,  ...,  0.0058,  0.8163, -0.5776]],
       device='cuda:0')